In [14]:
import pandas as pd
import sqlite3
from tqdm.notebook import tqdm

In [15]:
pd.set_option("display.max_columns", 500)

In [16]:
#Listing currently existing tables in the database
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
        SELECT name
        FROM sqlite_master 
        WHERE type ='table' 
        AND name NOT LIKE 'sqlite_%';
        """ 

    test_df = pd.read_sql(query, db)

display(test_df)

,name
0,taxonomy
1,hop_teaming
2,npidata
3,filtered_hop_teaming


In [17]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT COUNT(*) AS count_all
    FROM filtered_hop_teaming
    JOIN npi USING()
    WHERE entity_type_code;
    """ 
    
    test = pd.read_sql(query, db)

test

,count_all
0,131998


In [19]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM filtered_hop_teaming
    LIMIT 1;
    """ 
    
    test = pd.read_sql(query, db)

test

,from_npi,to_npi,patient_count,transaction_count,average_day_wait,std_day_wait
0,1043215882,1003028770,29,44,47.455,56.183


In [5]:
# Find NPIs with largest number of referrals
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT SUM(transaction_count) AS total_referrals, to_npi, n.provider_org_name
    FROM filtered_hop_teaming AS f
    JOIN npidata AS n
    ON f.to_npi = n.npi
    GROUP BY to_npi
    ORDER BY total_referrals DESC
    LIMIT 20;
    """ 
    
    test = pd.read_sql(query, db)

test

,total_referrals,to_npi,provider_org_name
0,1113176,1104202761,VANDERBILT UNIVERSITY MEDICAL CENTER
1,876679,1093741464,"ADVANCED DIAGNOSTIC IMAGING, PC"
2,849974,1437194669,SAINT THOMAS MEDICAL PARTNERS
3,764787,1396882205,VANDERBILT UNIVERSITY MEDICAL CENTER
4,711886,1861478489,RADIOLOGY ALLIANCE PC
5,642771,1003863580,"ASSOCIATED PATHOLOGISTS, LLC"
6,409468,1245393057,CENTENNIAL HEART LLC
7,382116,1235186800,"PATHGROUP LABS, LLC"
8,345431,1215932413,"ANESTHESIA MEDICAL GROUP, PC"
9,311019,1811955917,TENNESSEE ONCOLOGY PLLC


In [27]:
# Find competitor hosptitals with the largest number of total referrals.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT SUM(transaction_count) AS total_referrals, to_npi, n.provider_org_name, t.classification
    FROM filtered_hop_teaming AS f
    JOIN npidata AS n
    ON f.to_npi = n.npi
    JOIN taxonomy AS t
    USING(taxonomy_code)
    WHERE classification = 'Family Medicine'
    GROUP BY to_npi, provider_org_name
    --HAVING provider_org_name LIKE '%VUMC%'
    ORDER BY total_referrals DESC
    LIMIT 50;
    """ 
    
    test = pd.read_sql(query, db)

test

,total_referrals,to_npi,provider_org_name,classification
0,179540,1205886264,"MAURY REGIONAL MEDICAL GROUP, INC",Family Medicine
1,86293,1568464873,DICKSON MEDICAL ASSOCIATES PC,Family Medicine
2,20477,1295044097,"NORTHSIDE MEDICAL PROFESSIONALS, P.C.",Family Medicine
3,19298,1700873171,COLUMBIA MEDICAL GROUP-SOUTHERN HILLS INC,Family Medicine
4,19138,1467598870,TENNESSEE PHYSICIANS ALLIANCE PC,Family Medicine
5,15703,1255419255,SUMMIT PRIMARY CARE,Family Medicine
6,11518,1679677199,"CRIPPS, HOOPER & RHODY, PLLC",Family Medicine
7,11046,1093768137,LEBANON HMA PHYSICIAN MANAGEMENT LLC,Family Medicine
8,9714,1073986410,"BLUESKY HOUSECALLS, LLC",Family Medicine
9,8078,1437194263,MEDICAL GROUP - STONECREST FP INC,Family Medicine


In [26]:
# Find competitor hosptitals with the largest number of total referrals.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT classification, SUM(transaction_count)
    FROM filtered_hop_teaming AS f
    JOIN npidata AS n
    ON f.to_npi = n.npi
    JOIN taxonomy AS t
    USING(taxonomy_code)
    --WHERE provider_org_name LIKE '%VANDERBILT%'
    GROUP BY classification
    ORDER BY SUM(transaction_count) DESC;
    """ 
    
    test = pd.read_sql(query, db)

test

,classification,SUM(transaction_count)
0,Internal Medicine,3044556
1,General Acute Care Hospital,2032780
2,Radiology,1644510
3,Specialist,770769
4,Pathology,577485
...,...,...
63,Oral & Maxillofacial Surgery,139
64,Public Health or Welfare,77
65,"Dietitian, Registered",74
66,Medicare Defined Swing Bed Unit,62


In [8]:
# Find addresses of all Vanderbilt locations
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM npidata
    WHERE provider_org_name LIKE '%VANDERBILT%'
    GROUP BY provider_business_address_1
        , provider_business_city
        , provider_business_state
        , provider_business_zip5
    """ 
    
    test = pd.read_sql(query, db)

test

,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,taxonomy_code,provider_business_zip5
0,1104166313,2,"VANDERBILT MAURY RADIATION ONCOLOGY, LLC",None,None,None,None,None,None,1003 RESERVE BLVD,SUITE 120,SPRING HILL,TN,37174,261QX0203X,37174
1,1316974231,2,VANDERBILT UNIVERSITY,None,None,None,None,None,None,1161 21ST AVE S,ROOM 1815 THE VANDERBILT CLINIC,NASHVILLE,TN,372320001,3336S0011X,37232
2,1013215086,2,VANDERBILT UNIVERSITY,None,None,None,None,None,None,1161 21ST AVE S RM 1815,THE VANDERBILT CLINIC,NASHVILLE,TN,372320001,3336C0003X,37232
3,1467599563,2,VANDERBILT UNIVERSITY,None,None,None,None,None,None,"1161 21ST AVENUE, SOUTH",S-7400 MEDICAL CENTER NORTH,NASHVILLE,TN,37232,314000000X,37232
4,1891951513,2,CONCENTRA VANDERBILT LLC,None,None,None,None,None,None,1203-A MEMORIAL BLVD.,None,MURFREESBORO,TN,37129,261QH0100X,37129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
66,1972749406,2,VANDERBILT ORTHOPAEDIC INSTITUTE,None,None,None,None,None,None,MEDICAL CENTER EAST SOUTH TOWER,SUITE 3200,NASHVILLE,TN,372320001,283X00000X,37232
67,1992770119,2,VANDERBILT MEDICAL CENTER,None,None,None,None,None,None,S-3414 MEDICAL CENTER NORTH,C/O INTERNAL MEDICINE DEPT,NASHVILLE,TN,372320001,282N00000X,37232
68,1508132242,2,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,VANDERBILT CHILDRENS HOSPITAL,"2200 CHILDREN'S WAY, DOT 11126",NASHVILLE,TN,372320001,261Q00000X,37232
69,1245485200,2,VANDERBILT UNIVERSITY MEDICAL CENTER,None,None,None,None,None,None,VANDERBILT MEDICAL GENETICS,DD-2205 MCN,NASHVILLE,TN,372320001,170300000X,37232


In [9]:
# Find competitor hosptitals with the largest number of total referrals.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    WITH vandy_address AS (
        SELECT *
        FROM npidata
        WHERE provider_org_name LIKE '%VANDERBILT%'
        GROUP BY provider_business_address_1
            , provider_business_city
            , provider_business_state
            , provider_business_zip5
    )
    SELECT *
    FROM npidata
    --JOIN vandy_address AS va USING(npi)
    WHERE provider_org_name NOT LIKE '%VANDERBILT%'
    AND provider_business_address_1 IN (SELECT provider_business_address_1 FROM vandy_address)
    AND provider_business_city IN (SELECT provider_business_city FROM vandy_address)
    AND provider_business_zip5 IN (SELECT provider_business_zip5 FROM vandy_address)
    """ 
    
    test = pd.read_sql(query, db)

test

,npi,entity_type_code,provider_org_name,provider_last_name,provider_first_name,provider_middle_name,provider_name_prefix,provider_name_suffix,provider_credential,provider_business_address_1,provider_business_address_2,provider_business_city,provider_business_state,provider_business_zip,taxonomy_code,provider_business_zip5
0,1750387635,2,"TENNESSEE ADVANCED NEURODIAGNOSTIC INSTITUTE, ...",None,None,None,None,None,None,4525 HARDING RD,STE 102,NASHVILLE,TN,372052145,261QM1200X,37205
1,1568456986,2,TENNESSEE ORTHOPAEDIC ALLIANCE PA,None,None,None,None,None,None,3443 DICKERSON PIKE,SUITE 480,NASHVILLE,TN,372072519,207X00000X,37207
2,1134116510,2,ARTHRITIS SPECIALISTS OF NASHVILLE INC,None,None,None,None,None,None,3443 DICKERSON PIKE,SUITE 520,NASHVILLE,TN,372072519,207RR0500X,37207
3,1427039858,2,"NES TENNESSEE, INC.",None,None,None,None,None,None,2835 HIGHWAY 231 N,None,SHELBYVILLE,TN,371607327,207P00000X,37160
4,1790761492,2,METRO MEDICAL SKYLINE PHARMACY,None,None,None,None,None,None,3443 DICKERSON PIKE,SUITE 110,NASHVILLE,TN,372072519,183500000X,37207
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,1770002263,2,SHELBYVILLE HOSPITAL COMPANY LLC,None,None,None,None,None,None,1612 N MAIN ST STE A,None,SHELBYVILLE,TN,371602392,207Q00000X,37160
72,1952820433,2,SHELBYVILLE HOSPITAL COMPANY LLC,None,None,None,None,None,None,1401 MADISON ST,None,SHELBYVILLE,TN,371603629,207Q00000X,37160
73,1619496197,2,SHELBYVILLE HOSPITAL COMPANY LLC,None,None,None,None,None,None,1612 N MAIN ST STE B,None,SHELBYVILLE,TN,371602392,207Q00000X,37160
74,1699294173,2,SHELBYVILLE HOSPITAL COMPANY LLC,None,None,None,None,None,None,3335 HIGHWAY 41A N,None,UNIONVILLE,TN,371805033,207Q00000X,37180


In [13]:
# TESTING SQUARE
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT provider_business_state, COUNT(provider_business_state)
    FROM npidata
    GROUP BY provider_business_state
    """ 
    
    test = pd.read_sql(query, db)

test

,provider_business_state,COUNT(provider_business_state)
0,FL,1
1,IA,1
2,IN,3
3,KY,1
4,NE,2
5,NH,73
6,NJ,1
7,NM,1
8,OR,1
9,QOM,1


In [ ]:
# TESTING SQUARE
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM npidata
    LIMIT 1;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Tingting's code for exporting to Neo4j

nodes = list(set(ht.from_npi.tolist() + ht.to_npi.tolist()))
node_df = pd.DataFrame({'npi:ID': nodes})
node_df[':LABEL'] = "Provider"
node_df.to_csv('import/nodes.csv', index = False)
edges = pd.DataFrame({':START_ID' : ht.from_npi, 'patient_count': ht.patient_count, 
                      'transaction_count': ht.transaction_count, 
                     ':END_ID' : ht.to_npi})
edges[':TYPE'] = 'REFERRED_TO'
edges.to_csv('import/edges.csv', index = False)

In [ ]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT *
    FROM taxonomy
    LIMIT 1;
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT DISTINCT classification
    FROM taxonomy
    """ 
    
    test = pd.read_sql(query, db)

test

In [ ]:
# Test to confirm the tables loaded.
with sqlite3.connect('../data/hcbb.sqlite') as db :
    query = """
    SELECT npidata.*
    FROM npidata
    JOIN taxonomy USING (taxonomy_code)
    WHERE classification = 'train'
    """ 
    
    test = pd.read_sql(query, db)

test